# MaterialVision: Embedding Pipeline

This notebook provides a streamlined approach to:
1. Load models and datasets
2. Load validation data and create embeddings
3. Generate embeddings for custom text samples

In [1]:
# Essential imports and setup
import sys
import os
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
import pickle

# Import model functions
from models import (
    load_clipp_scibert,
    load_clipp_distilbert, 
    load_mobileclip,
    load_blip
)

# Available models
MODELS = ['CLIPP-SciBERT', 'CLIPP-DistilBERT', 'MobileCLIP', 'BLIP']

print("✅ Imports completed")
print(f"📋 Available models: {MODELS}")

/home/jipengsun/.conda/envs/clipp/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/jipengsun/.conda/envs/clipp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Adding to path: /home/jipengsun/MaterialVision/models/CLIPP_allenai
✅ Successfully imported CLIPP SciBERT
Adding to path: /home/jipengsun/MaterialVision/models/CLIPP_bert
✅ Successfully imported CLIPP SciBERT
Adding to path: /home/jipengsun/MaterialVision/models/CLIPP_bert
✅ Successfully imported CLIPP DistilBERT
Adding to path: /home/jipengsun/MaterialVision/models/Apple_MobileCLIP
✅ Successfully imported MobileCLIP
Adding to path: /home/jipengsun/MaterialVision/models/Salesforce
❌ Error importing BLIP: No module named 'models.CLIPP_bert'; 'models' is not a package
✅ Imports completed
📋 Available models: ['CLIPP-SciBERT', 'CLIPP-DistilBERT', 'MobileCLIP', 'BLIP']
✅ Successfully imported CLIPP DistilBERT
Adding to path: /home/jipengsun/MaterialVision/models/Apple_MobileCLIP
✅ Successfully imported MobileCLIP
Adding to path: /home/jipengsun/MaterialVision/models/Salesforce
❌ Error importing BLIP: No module named 'models.CLIPP_bert'; 'models' is not a package
✅ Imports completed
📋 Availa

In [6]:
# Define validation data path
VAL_CSV = Path('../data/alpaca_mbj_bandgap_test.csv')
BATCH_SIZE = 32

print(f"Validation CSV path: {VAL_CSV}")
print(f"Validation CSV exists: {VAL_CSV.exists()}")
# Load validation data
if VAL_CSV.exists():
    val_df = pd.read_csv(VAL_CSV)
    print(f"✅ Loaded validation data with {len(val_df)} samples")
    print(f"   Columns: {list(val_df.columns)}")
else:
    print(f"❌ Validation CSV not found at {VAL_CSV}")
    val_df = None

Validation CSV path: ../data/alpaca_mbj_bandgap_test.csv
Validation CSV exists: True
✅ Loaded validation data with 1000 samples
   Columns: ['instruction', 'input', 'response', 'id', 'image']
✅ Loaded validation data with 1000 samples
   Columns: ['instruction', 'input', 'response', 'id', 'image']


In [7]:
def create_text_embeddings(model_name, texts):
    """
    Create  embeddings for given sample using specified model.
    
    Args:
        model_name: Name of the model to use
        texts: Single text string or list of texts
    
    Returns:
        numpy array of embeddings
    """
    if isinstance(texts, str):
        texts = [texts]
    
    print(f"🔄 Loading {model_name} model...")
    
    try:
        # Define device
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        # Load model based on type
        if model_name == 'CLIPP-SciBERT':
            checkpoint_path = '../models/CLIPP_allenai/checkpoints/best_clipp.pth'
            model, tokenizer, _ = load_clipp_scibert(checkpoint_path, device)
            
            embeddings = []
            model.eval()
            with torch.no_grad():
                for text in texts:
                    tokens = tokenizer(text, padding=True, truncation=True, 
                                     return_tensors="pt", max_length=512).to(device)
                    text_features = model.get_text_features(tokens['input_ids'], tokens['attention_mask'])
                    embeddings.append(text_features.cpu().numpy())
        
        elif model_name == 'CLIPP-DistilBERT':
            checkpoint_path = '../models/CLIPP_bert/checkpoints/best_clipp_bert.pth'
            model, tokenizer, _ = load_clipp_distilbert(checkpoint_path, device)
            
            embeddings = []
            model.eval()
            with torch.no_grad():
                for text in texts:
                    tokens = tokenizer(text, padding=True, truncation=True, 
                                     return_tensors="pt", max_length=512).to(device)
                    text_features = model.get_text_features(tokens['input_ids'], tokens['attention_mask'])
                    embeddings.append(text_features.cpu().numpy())
        
        elif model_name == 'MobileCLIP':
            import open_clip
            checkpoint_path = '../models/Apple_MobileCLIP/checkpoints/best_clipp_apple.pth'
            model, tokenizer, _ = load_mobileclip(checkpoint_path, device)
            
            embeddings = []
            model.eval()
            with torch.no_grad():
                for text in texts:
                    tokens = open_clip.tokenize([text]).to(device)
                    text_features = model.get_text_features(tokens)
                    embeddings.append(text_features.cpu().numpy())
        
        elif model_name == 'BLIP':
            checkpoint_path = '../models/Salesforce/checkpoints_blip/best_blip.pth'
            model, processor, _ = load_blip(checkpoint_path, device)
            
            embeddings = []
            model.eval()
            with torch.no_grad():
                for text in texts:
                    inputs = processor(text=[text], return_tensors="pt", 
                                     padding=True, truncation=True, max_length=512).to(device)
                    text_embeds = model.get_text_features(**inputs)
                    text_embeds = F.normalize(text_embeds, p=2, dim=1)
                    embeddings.append(text_embeds.cpu().numpy())
        
        result = np.vstack(embeddings) if len(embeddings) > 1 else embeddings[0]
        print(f"✅ Generated embeddings: {result.shape}")
        return result.squeeze() if len(texts) == 1 else result
        
    except Exception as e:
        print(f"❌ Error with {model_name}: {e}")
        import traceback
        traceback.print_exc()
        return None

print("✅ Text embedding function ready")

✅ Text embedding function ready


In [9]:
# Generate embeddings for validation dataset
def process_validation_data(models_to_use=None):
    """
    Process validation dataset and create embeddings for all models.
    """
    if val_df is None:
        print("❌ No validation data available")
        return {}
    
    models_to_process = models_to_use or MODELS
    results = {}
    
    # Sample first 10 texts for demo (change as needed)
    sample_texts = val_df['input'].head(10).tolist()
    print(f"📊 Processing {len(sample_texts)} validation samples")
    
    for model_name in models_to_process:
        print(f"\n🔄 Processing {model_name}...")
        embeddings = create_text_embeddings(model_name, sample_texts)
        
        if embeddings is not None:
            results[model_name] = {
                'embeddings': embeddings,
                'texts': sample_texts,
                'shape': embeddings.shape
            }
            print(f"✅ {model_name}: {embeddings.shape}")
        else:
            print(f"❌ {model_name}: Failed")
    
    return results

# Run validation processing
print("🚀 Starting validation data processing...")
validation_results = process_validation_data()
print(f"\n🎉 Completed processing for {len(validation_results)} models")

🚀 Starting validation data processing...
📊 Processing 10 validation samples

🔄 Processing CLIPP-SciBERT...
🔄 Loading CLIPP-SciBERT model...


2025-11-09 19:57:08,204 INFO: Loading pretrained weights from Hugging Face hub (timm/vit_base_patch16_224.augreg2_in21k_ft_in1k)
2025-11-09 19:57:08,245 INFO: [timm/vit_base_patch16_224.augreg2_in21k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2025-11-09 19:57:08,245 INFO: [timm/vit_base_patch16_224.augreg2_in21k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


✅ Generated embeddings: (10, 256)
✅ CLIPP-SciBERT: (10, 256)

🔄 Processing CLIPP-DistilBERT...
🔄 Loading CLIPP-DistilBERT model...


2025-11-09 19:57:11,817 INFO: Loading pretrained weights from Hugging Face hub (timm/resnet50.a1_in1k)
2025-11-09 19:57:11,860 INFO: [timm/resnet50.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2025-11-09 19:57:11,860 INFO: [timm/resnet50.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2025-11-09 19:57:13,044 INFO: Loaded MobileCLIP-S2 model config.
2025-11-09 19:57:13,044 INFO: Loaded MobileCLIP-S2 model config.


✅ Generated embeddings: (10, 256)
✅ CLIPP-DistilBERT: (10, 256)

🔄 Processing MobileCLIP...
🔄 Loading MobileCLIP model...


2025-11-09 19:57:14,404 INFO: Loading pretrained MobileCLIP-S2 weights (datacompdr).


✅ Generated embeddings: (10, 256)
✅ MobileCLIP: (10, 256)

🔄 Processing BLIP...
🔄 Loading BLIP model...
✅ Generated embeddings: (10, 256)
✅ BLIP: (10, 256)

🎉 Completed processing for 4 models
✅ Generated embeddings: (10, 256)
✅ BLIP: (10, 256)

🎉 Completed processing for 4 models


In [11]:
# Test with custom text samples
def test_custom_text(text_input, models_to_use=None):
    """
    Generate embeddings for custom text using all or specified models.
    
    Args:
        text_input: Single text or list of texts
        models_to_use: List of model names (None for all)
    
    Returns:
        Dictionary of model_name -> embeddings
    """
    if isinstance(text_input, str):
        print(f"📝 Input text: \"{text_input[:100]}...\")")
    else:
        print(f"📝 Processing {len(text_input)} texts")
    
    models_to_process = models_to_use or MODELS
    results = {}
    
    for model_name in models_to_process:
        print(f"\n🔄 {model_name}...")
        embeddings = create_text_embeddings(model_name, text_input)
        
        if embeddings is not None:
            results[model_name] = embeddings
            norm = np.linalg.norm(embeddings) if embeddings.ndim == 1 else [np.linalg.norm(emb) for emb in embeddings]
            print(f"✅ Shape: {embeddings.shape}, Norm: {norm}")
    
    return results

# Example usage with custom texts
print("\n🧪 Testing with custom material descriptions...")

# Test single text
custom_text = "Silicon carbide semiconductor with high thermal conductivity for power electronics"
single_results = test_custom_text(custom_text)

# Test multiple texts
custom_texts = [
    "Graphene-based composite for energy storage applications",
    "Perovskite solar cell with enhanced stability",
    "Titanium alloy with superior mechanical properties"
]
batch_results = test_custom_text(custom_texts)

print("\n🎉 Custom text testing completed!")


🧪 Testing with custom material descriptions...
📝 Input text: "Silicon carbide semiconductor with high thermal conductivity for power electronics...")

🔄 CLIPP-SciBERT...
🔄 Loading CLIPP-SciBERT model...


2025-11-09 19:58:55,817 INFO: Loading pretrained weights from Hugging Face hub (timm/vit_base_patch16_224.augreg2_in21k_ft_in1k)
2025-11-09 19:58:55,869 INFO: [timm/vit_base_patch16_224.augreg2_in21k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2025-11-09 19:58:55,869 INFO: [timm/vit_base_patch16_224.augreg2_in21k_ft_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.


✅ Generated embeddings: (1, 256)
✅ Shape: (256,), Norm: 1.0

🔄 CLIPP-DistilBERT...
🔄 Loading CLIPP-DistilBERT model...


2025-11-09 19:58:59,370 INFO: Loading pretrained weights from Hugging Face hub (timm/resnet50.a1_in1k)
2025-11-09 19:58:59,414 INFO: [timm/resnet50.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2025-11-09 19:58:59,414 INFO: [timm/resnet50.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2025-11-09 19:59:00,544 INFO: Loaded MobileCLIP-S2 model config.
2025-11-09 19:59:00,544 INFO: Loaded MobileCLIP-S2 model config.


✅ Generated embeddings: (1, 256)
✅ Shape: (256,), Norm: 1.0

🔄 MobileCLIP...
🔄 Loading MobileCLIP model...


...

✅ Generated embeddings: (3, 256)
✅ Shape: (3, 256), Norm: [np.float32(1.0), np.float32(1.0), np.float32(0.99999994)]

🔄 CLIPP-DistilBERT...
🔄 Loading CLIPP-DistilBERT model...


2025-11-09 19:59:13,680 INFO: Loading pretrained weights from Hugging Face hub (timm/resnet50.a1_in1k)
2025-11-09 19:59:13,721 INFO: [timm/resnet50.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2025-11-09 19:59:13,721 INFO: [timm/resnet50.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2025-11-09 19:59:14,814 INFO: Loaded MobileCLIP-S2 model config.
2025-11-09 19:59:14,814 INFO: Loaded MobileCLIP-S2 model config.


✅ Generated embeddings: (3, 256)
✅ Shape: (3, 256), Norm: [np.float32(1.0), np.float32(1.0), np.float32(1.0)]

🔄 MobileCLIP...
🔄 Loading MobileCLIP model...


2025-11-09 19:59:16,350 INFO: Loading pretrained MobileCLIP-S2 weights (datacompdr).


✅ Generated embeddings: (3, 256)
✅ Shape: (3, 256), Norm: [np.float32(1.0), np.float32(1.0), np.float32(1.0)]

🔄 BLIP...
🔄 Loading BLIP model...
✅ Generated embeddings: (3, 256)
✅ Shape: (3, 256), Norm: [np.float32(0.99999994), np.float32(1.0), np.float32(1.0)]

🎉 Custom text testing completed!
✅ Generated embeddings: (3, 256)
✅ Shape: (3, 256), Norm: [np.float32(0.99999994), np.float32(1.0), np.float32(1.0)]

🎉 Custom text testing completed!


In [ ]:
# Save results (optional)
def save_results(results, filename='embedding_results.pkl'):
    """
    Save embedding results to pickle file.
    """
    save_path = Path('./outputs') 
    save_path.mkdir(exist_ok=True)
    
    filepath = save_path / filename
    with open(filepath, 'wb') as f:
        pickle.dump(results, f)
    
    print(f"💾 Results saved to {filepath}")
    return filepath

# Save validation and custom results
all_results = {
    'validation_results': validation_results,
    'single_text_results': single_results,
    'batch_text_results': batch_results
}

save_results(all_results, 'simple_embedding_results.pkl')

print("\n📊 Summary:")
print(f"✅ Validation models processed: {len(validation_results)}")
print(f"✅ Single text models: {len(single_results)}")
print(f"✅ Batch text models: {len(batch_results)}")
print("\n🎯 Pipeline completed successfully!")

## Quick Usage Examples

### Text Embeddings
```python
# Generate embeddings for any text
my_text = "Your material description here"
results = test_custom_text(my_text, models_to_use=['CLIPP-SciBERT', 'MobileCLIP'])

# Generate text embeddings directly
text_embeddings = create_text_embeddings('CLIPP-SciBERT', "Your text here")
```

### Image Embeddings
```python
# Generate embeddings for images
image_paths = ["/path/to/image1.jpg", "/path/to/image2.jpg"]
image_embeddings = create_image_embeddings('CLIPP-SciBERT', image_paths)

# Single image
single_image_embedding = create_image_embeddings('CLIPP-SciBERT', "/path/to/image.jpg")
```

### Multimodal Embeddings
```python
# Generate both text and image embeddings
results = test_custom_multimodal(
    text_input=["Material description 1", "Material description 2"],
    image_paths=["/path/to/image1.jpg", "/path/to/image2.jpg"],
    models_to_use=['CLIPP-SciBERT']
)

# Text and images together
multimodal_embeddings = create_multimodal_embeddings(
    model_name='CLIPP-SciBERT',
    texts=["Silicon carbide semiconductor"],
    image_paths=["/path/to/sic_image.jpg"]
)
```

### Process Validation Data
```python
# Process validation data with specific models (text + images)
val_results = process_validation_data(models_to_use=['CLIPP-SciBERT'], include_images=True)

# Text only
val_results_text = process_validation_data(models_to_use=['CLIPP-SciBERT'], include_images=False)
```